In [109]:
import pandas as pd
df = pd.read_csv("antennes.csv")

In [110]:
print(df.shape)

In [111]:
df.columns

### Checking for Duplicate Rows

In [112]:
df.duplicated().sum()

In [113]:
df.duplicated(subset=["code_site"]).sum()

### Standardizing Operator Names

In [114]:
df["operateur"].unique()

In [115]:
df['operateur'] = df['operateur'].str.upper()
df["operateur"].unique()

### Checking for Missing Values

In [116]:
missing_count = df.isnull().sum()
missing_percentage = (df.isnull().sum() / len(df)) * 100

missing_data = pd.DataFrame({
    'Missing Count': missing_count,
    'Missing Percentage (%)': missing_percentage.round(2)
})

print(missing_data)

### Dropping the column with all missing values

In [117]:
df = df.drop('mise_en_serv_5g_700', axis=1)

### Converting columns to Datetime

In [118]:
date_cols = [
    "mise_en_serv",
    "mise_en_serv_4g",
    "mise_en_serv_5g_3500",
]

df[date_cols].dtypes


In [119]:
for col in date_cols:
    df[col] = pd.to_datetime(df[col], errors="coerce") # NaT

In [120]:
df[date_cols].dtypes

### Identifying incoherent dates

In [121]:
df[
    (df["mise_en_serv_5g_3500"] < df["mise_en_serv_4g"]) |
    (df["mise_en_serv_4g"] < df["mise_en_serv"])
    ]


In [122]:
anomalies = (
        (df["mise_en_serv_5g_3500"] < df["mise_en_serv_4g"]) |
        (df["mise_en_serv_4g"] < df["mise_en_serv"])
)
pourcentage_anomalies = anomalies.mean() * 100
print(f"{pourcentage_anomalies:.2f}% des lignes ont des anomalies de dates")

### Replace these incorrect dates with NaT 

In [123]:
df.loc[df['mise_en_serv_4g'] < df['mise_en_serv'], 'mise_en_serv_4g'] = pd.NaT
df.loc[df['mise_en_serv_5g_3500'] < df['mise_en_serv_4g'], 'mise_en_serv_5g_3500'] = pd.NaT

### Handling missing values in the type column

In [124]:
df["type"].unique()

In [125]:
import numpy as np

df["missing_type"] = np.where(df["type"].isna(), 1, 0)
df["missing_mise_en_serv"] = np.where(df["mise_en_serv"].isna(), 1, 0)
df["missing_mise_en_serv_4g"] = np.where(df["mise_en_serv_4g"].isna(), 1, 0)
df["missing_mise_en_serv_5g_3500"] = np.where(df["mise_en_serv_5g_3500"].isna(), 1, 0)

columns_of_interest = [
    "missing_type",
    "missing_mise_en_serv",
    "missing_mise_en_serv_4g",
    "missing_mise_en_serv_5g_3500",
]


In [126]:
import seaborn as sns
import matplotlib.pyplot as plt

correlation_matrix = df[columns_of_interest].corr(method="spearman")

sns.set_theme(style="dark")
plt.figure(figsize=(8, 6))
heatmap = sns.heatmap(
    correlation_matrix, annot=True, fmt=".2f", cmap="coolwarm",
    cbar_kws={"label": "Spearman correlation"}
)
heatmap.set_title("Correlation heatmap des valeurs manquantes")
plt.show()

### Analysis of missing value correlations

From this correlation heatmap, we can make several observations about missing values:

   - `missing_type` has a fairly strong correlation with `missing_mise_en_serv_4g` (0.60) and with `missing_mise_en_serv_5g_3500` (0.46).  
   - This confirms that when the `type` column is missing, the 4G and 5G deployment dates are often missing as well.

   - `missing_mise_en_serv_4g` and `missing_mise_en_serv_5g_3500` have a strong correlation of 0.77.  
   - This indicates that if the 4G deployment date is missing, the 5G deployment date is likely missing too.
      
=> We cannot infer `type` values from these columns

In [127]:
missing_counts = df[df["type"].isna()].groupby("operateur").size()
total_counts = df.groupby("operateur").size()

percent = (missing_counts / total_counts * 100).sort_values()

plt.figure(figsize=(8,6))
sns.barplot(x=percent.index, y=percent.values)
plt.ylabel("Pourcentage de valeurs 'type' manquantes")
plt.xlabel("Opérateur")
plt.title("Taux de valeurs 'type' manquantes par opérateur")
plt.xticks(rotation=45)
plt.show()


=> MNAR missing not at random


### Imputation with constant value 

In [128]:
df["type_clean"] = df["type"].fillna("Unknown")


### Delete useless columns

In [129]:
df.columns


In [130]:
del_col = [
    'missing_type',
    'missing_mise_en_serv',
    'missing_mise_en_serv_4g',
    'missing_mise_en_serv_5g_3500',
    'type'

]

df = df.drop(columns=del_col)

In [131]:
df.columns

In [132]:
df["geo_point_2d"].dtypes


### geo_shape and geo_point_2d 
These 2 columns contain the same information but in different formats i am going to keep the second to convert it simply to postgis format (wkt) 

In [133]:
import ast 

def extract_lat_lon(geo_str):
    geo_dict = ast.literal_eval(geo_str)  
    return pd.Series([geo_dict['lat'], geo_dict['lon']])

df[['lat','lon']] = df['geo_point_2d'].apply(extract_lat_lon)


In [134]:
df['geom'] = df.apply(lambda row: f"POINT({row['lon']} {row['lat']})", axis=1)


In [135]:
df

In [136]:
df = df.drop(columns=['geo_point_2d', 'geo_shape','lat','lon'])


### Extract arrondissement value from the ardt column

In [137]:
df["ardt"].dtypes

In [138]:
df['ardt'].unique()

In [139]:
df['ardt'].astype(str).str.startswith('76').sum()


=> an aberrant value 

In [140]:
df[df['ardt'].astype(str).str.startswith('76')]

In [141]:
df = df[df['ardt'].astype(str).str.startswith('75')].copy()

In [142]:
df['arrondissement'] = df['ardt'].astype(str).str[-2:].astype(int)
df = df.drop(columns=['ardt'])
df

In [143]:
df.to_csv("antennes_clean.csv", index=False)

### Load cleaned data to postgres + postgis

In [144]:
from db_utils import load_to_db
load_to_db(df)